In [14]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
#os.chdir('C:\\Users\\Administrator\\Desktop\\Prudhvi\\Prudhvi\\Data Science\\Data')
os.chdir('C:\\Users\\Administrator\\Desktop\\Prudhvi\\Prudhvi\\Data Science\\Data')

In [15]:
pd.options.display.max_columns = None
#Home
#telco=pd.read_csv('Telco_Customer_Attr.csv')
#Office
telco=pd.read_csv('Teleco_Cust_Attr.csv')
telco.head()
#print(telco.shape)

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,Yes,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,No,Yes,No,No,No,One year,No,Mailed check,56.95,1889.50,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,Yes,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,No,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


In [16]:
Mean_No_TotalCharges=telco.groupby('Churn')['TotalCharges'].mean()['No']
Mean_Yes_TotalCharges=telco.groupby('Churn')['TotalCharges'].mean()['Yes']

In [17]:
telco.loc[(telco['TotalCharges'].isnull()) & (telco['Churn']=='No'),['TotalCharges']]=Mean_No_TotalCharges
telco.loc[(telco['TotalCharges'].isnull()) & (telco['Churn']=='Yes'),['TotalCharges']]=Mean_Yes_TotalCharges

In [18]:
telco.drop(['customerID'],axis=1,inplace=True)

In [19]:
telco['SeniorCitizen']=telco['SeniorCitizen'].astype('object')

In [20]:
telco_backup=telco

In [21]:
y=telco['Churn']
telco=telco.drop('Churn',axis=1)

In [22]:
from sklearn.preprocessing import LabelBinarizer
from collections import defaultdict
import pandas as pd
encoding=defaultdict(LabelBinarizer)
columns_nominal=['gender','Partner','Dependents','PhoneService','MultipleLines','InternetService','OnlineSecurity','OnlineBackup','DeviceProtection','TechSupport','StreamingTV','StreamingMovies','PaperlessBilling','PaymentMethod']
for i in columns_nominal:
    encoded_df=pd.DataFrame()
    d=pd.DataFrame(encoding[i].fit_transform(telco[i]))
    test_column=d.columns.values
    list1=[i+'_'+str(j) for j in test_column]
    d.columns=list1
    encoded_df=pd.concat([encoded_df,d],axis=1)
    telco.drop(i,axis=1, inplace=True)
    telco=pd.concat([telco,encoded_df],axis=1)

In [23]:
from sklearn.preprocessing import LabelEncoder
lb_make = LabelEncoder()
telco['Contract']=lb_make.fit_transform(telco["Contract"])

In [29]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(telco,y,test_size=0.30,random_state=42)

### Performing Grid Search to find the right C

In [68]:
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
LR=LogisticRegression()
parameters={'C':[0.1,0.01,0.001],'max_iter':[100,150,200,500]}
GSCV_LR=GridSearchCV(LR,parameters)
GSCV_LR.fit(X_train,y_train)
print('Best Parameters: \n',GSCV_LR.best_params_)

Best Parameters: 
 {'C': 0.1, 'max_iter': 100}


In [73]:
from sklearn.metrics import confusion_matrix
C_values=[0.05,0.1,0.01,0.001,0.0001]
for c in C_values:
    LR=LogisticRegression(C=c)
    LR.fit(X_train,y_train)
    Predictions_train=LR.predict(X_train)
    print('Confusion matrix for train:',c,'\n',confusion_matrix(y_train,Predictions_train))
    Predictions_test=LR.predict(X_test)
    print('Confusion matrix for train:',c,'\n',confusion_matrix(y_test,Predictions_test))

Confusion matrix for train: 10 
 [[3275  360]
 [ 596  699]]
Confusion matrix for train: 10 
 [[1389  150]
 [ 249  325]]
Confusion matrix for train: 100 
 [[3275  360]
 [ 596  699]]
Confusion matrix for train: 100 
 [[1389  150]
 [ 249  325]]
Confusion matrix for train: 1000 
 [[3275  360]
 [ 596  699]]
Confusion matrix for train: 1000 
 [[1389  150]
 [ 249  325]]
Confusion matrix for train: 10000 
 [[3275  360]
 [ 596  699]]
Confusion matrix for train: 10000 
 [[1389  150]
 [ 249  325]]


In [71]:
from sklearn.linear_model import LogisticRegression
LogClf=LogisticRegression(penalty='l1')
LogClf.fit(X_train,y_train)
LogClf_Predictions_train=LogClf.predict(X_train)

In [ ]:
## Important Features

In [72]:
LogClf.coef_

array([[ 1.98513785e-01, -5.46210015e-02, -7.12635281e-01,
        -1.62620809e-03,  2.86060245e-04, -7.00048258e-02,
         0.00000000e+00, -1.07198159e-01, -3.18021510e-01,
        -2.38590115e-01,  0.00000000e+00,  0.00000000e+00,
        -2.39085281e-01,  5.04284221e-01,  0.00000000e+00,
         5.74210517e-02, -3.60640366e-04, -3.83798430e-01,
         5.89844631e-02, -2.16632033e-04, -1.34966872e-01,
         0.00000000e+00, -6.51318736e-01,  0.00000000e+00,
         2.98938801e-01,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  7.49766077e-02,
        -7.57645792e-04,  0.00000000e+00,  2.95670290e-01,
         3.01270466e-01,  0.00000000e+00, -6.15532276e-02,
         3.23948973e-01, -4.50249800e-02]])

In [59]:
LogClf.max_iter

100